### Mask Classification Refactoring
22년 여름 방학에 딥러닝을 갓 배우고, 만들었던 마스크 분류기를 리팩토링

하게 된 이유 : 그 동안 많은 경험을 위해 닥치는 대로 딥러닝을 사용해보고, 여러 지식을 습득하기 바밨었으나, 이제는 이전 작품을 다시 리팩토링 및 코드를 최대한 gpt의 도움 없이 짜보면서 내실을 다지고 싶었기 때문.

In [ ]:
import torch
import os, glob